## Topic Modeling with Gensim
Topic Modeling is a technique to extract the hidden topics from large volumes of text. Latent Dirichlet Allocation(LDA) is a popular algorithm for topic modeling with excellent implementations in the Python’s Gensim package.

ref: https://www.machinelearningplus.com/nlp/ 

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.corpus import wordnet


import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
from pprint import pprint

np.random.seed(2018)

In [15]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatization(texts):
    # Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    texts_out = []
    for sent in texts:
        texts_out.append([lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in sent])
    return texts_out

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [16]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
#pprint(stop_words)

In [18]:
headers = pd.read_csv('abcnews-2017.csv', error_bad_lines=False);

In [19]:
headers[:5]

,headline_text,date
0,1986 queensland cabinet documents released,20170101
1,actor william cristopher mash chaplain dead at 84,20170101
2,adelaide hip hop artist dreams becoming influe...,20170101
3,adelaide's new year's fireworks,20170101
4,adelaides new years fireworks,20170101


### Data Preprocessing

#### Lemmatize example

In [20]:
# Convert to list
data = headers.headline_text.values.tolist()
pprint(data[:1])

['1986 queensland cabinet documents released']


## Tokenize words

In [21]:
data_words = list(sent_to_words(data))
pprint(data_words[:2])

[['queensland', 'cabinet', 'documents', 'released'],
 ['actor', 'william', 'cristopher', 'mash', 'chaplain', 'dead', 'at']]


## Remove Stopwords

In [22]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
pprint(data_words_nostops[:2])

[['queensland', 'cabinet', 'documents', 'released'],
 ['actor', 'william', 'cristopher', 'mash', 'chaplain', 'dead']]


In [23]:
data_lemmatized = lemmatization(data_words_nostops)
pprint(data_lemmatized[:2])

[['queensland', 'cabinet', 'document', 'release'],
 ['actor', 'william', 'cristopher', 'mash', 'chaplain', 'dead']]


## Create the Dictionary and Corpus for Topic Modeling

In [24]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
pprint(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [25]:
# If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.
id2word[0]

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('cabinet', 1), ('document', 1), ('queensland', 1), ('release', 1)]]

## Building the Topic Model

In [26]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## List the topics in LDA model

In [27]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.058*"australia" + 0.034*"charge" + 0.029*"nsw" + 0.020*"report" + '
  '0.018*"murder" + 0.016*"south" + 0.015*"bank" + 0.015*"test" + '
  '0.013*"scorecentre" + 0.013*"canberra"'),
 (1,
  '0.050*"say" + 0.050*"police" + 0.030*"man" + 0.022*"take" + 0.019*"cup" + '
  '0.017*"queensland" + 0.017*"show" + 0.014*"rise" + 0.013*"car" + '
  '0.013*"two"'),
 (2,
  '0.034*"new" + 0.025*"woman" + 0.019*"year" + 0.017*"world" + 0.016*"plan" + '
  '0.016*"state" + 0.013*"first" + 0.011*"house" + 0.011*"dy" + '
  '0.010*"victoria"'),
 (3,
  '0.027*"fire" + 0.022*"u" + 0.018*"may" + 0.016*"afl" + 0.016*"found" + '
  '0.015*"help" + 0.013*"high" + 0.013*"melbourne" + 0.013*"find" + '
  '0.012*"uk"'),
 (4,
  '0.043*"sydney" + 0.039*"attack" + 0.034*"could" + 0.033*"court" + '
  '0.023*"change" + 0.021*"hospital" + 0.021*"go" + 0.018*"dog" + '
  '0.014*"record" + 0.012*"country"'),
 (5,
  '0.025*"fear" + 0.025*"news" + 0.025*"end" + 0.019*"put" + 0.019*"concern" + '
  '0.018*"war" + 0.017*"c

## Coherence Score

In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.096813106869256

Coherence Score:  0.4989343331617513


## Visualize the topics-keywords

In [33]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/ubuntu/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.398822 -0.073840       1        1  14.148230
3     -0.131780 -0.373026       2        1  13.216247
7     -0.071789  0.105453       3        1  11.713610
1     -0.047664  0.150050       4        1  11.467966
9     -0.020465  0.044348       5        1  10.283163
0     -0.034530  0.053147       6        1  10.123476
6     -0.032627  0.027795       7        1   9.204846
4     -0.021107  0.026408       8        1   7.844595
8     -0.016061  0.018595       9        1   6.536467
5     -0.022800  0.021070      10        1   5.461400, topic_info=     Category        Freq        Term       Total  loglift  logprob
12    Default  839.000000   australia  839.000000  30.0000  30.0000
313   Default  752.000000  australian  752.000000  29.0000  29.0000
49    Default  823.000000         say  823.000000  28.0000  28.0000
48    Default  820.000000      police  820.000000  27.0000  27.0000
95    Default  571.000000       trump  571.000000  26.0000  26.0000
19    Default  685.000000         new  685.000000  25.0000  25.0000
274   Default  539.000000          wa  539.000000  24.0000  24.0000
231   Default  479.000000      sydney  479.000000  23.0000  23.0000
70    Default  491.000000      charge  491.000000  22.0000  22.0000
22    Default  434.000000      attack  434.000000  21.0000  21.0000
471   Default  487.000000         man  487.000000  20.0000  20.0000
28    Default  501.000000        fire  501.000000  19.0000  19.0000
213   Default  466.000000          sa  466.000000  18.0000  18.0000
301   Default  504.000000       woman  504.000000  17.0000  17.0000
427   Default  381.000000       could  381.000000  16.0000  16.0000
158   Default  424.000000         nsw  424.000000  15.0000  15.0000
219   Default  446.000000  government  446.000000  14.0000  14.0000
709   Default  365.000000       court  365.000000  13.0000  13.0000
1690  Default  365.000000        june  365.000000  12.0000  12.0000
562   Default  399.000000      school  399.000000  11.0000  11.0000
981   Default  420.000000           u  420.000000  10.0000  10.0000
256   Default  363.000000         win  363.000000   9.0000   9.0000
334   Default  355.000000        take  355.000000   8.0000   8.0000
178   Default  358.000000       perth  358.000000   7.0000   7.0000
20    Default  393.000000        year  393.000000   6.0000   6.0000
250   Default  351.000000        call  351.000000   5.0000   5.0000
10    Default  273.000000    adelaide  273.000000   4.0000   4.0000
458   Default  264.000000       crash  264.000000   3.0000   3.0000
84    Default  312.000000       death  312.000000   2.0000   2.0000
1070  Default  253.000000        face  253.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
1211  Topic10  193.089920        fear  194.059753   2.9025  -3.6979
618   Topic10  191.839890        news  192.809631   2.9024  -3.7044
590   Topic10  191.474548         end  192.444351   2.9024  -3.7063
1543  Topic10  149.493729         put  150.463531   2.9010  -3.9538
56    Topic10  144.276474     concern  145.246170   2.9008  -3.9893
61    Topic10  137.295929         war  138.265594   2.9004  -4.0389
247   Topic10  131.591324     council  132.561096   2.9001  -4.0813
1809  Topic10  130.138306    sentence  131.108017   2.9000  -4.0924
946   Topic10  109.835068      flight  110.804764   2.8987  -4.2620
2434  Topic10  102.231583        paul  103.201408   2.8980  -4.3338
1710  Topic10  101.119240    building  102.088966   2.8979  -4.3447
1748  Topic10   95.533112         abc   96.502754   2.8974  -4.4015
1031  Topic10   94.217720      worker   95.187447   2.8972  -4.4154
1484  Topic10   90.168671     chinese   91.138412   2.8968  -4.4593
58    Topic10   86.534767        post   87.504723   2.8963  -4.5005
1029  Topic10   85.241928       leave   86.211678   2.8962  -4.5155
129   Topic10   84.400627    security   85.37